In [6]:
import pandas as pd
import nltk
import csv
import json
nltk.download('punkt_tab')

from examples.FActScore.factscore.factscorer import FactScorer

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
!pip install torch -U

In [17]:
!python -m examples.FActScore.factscore.download_data

[Already exists] Skipping demos.zip
If you want to download the file in another location, please specify a different path
[Already exists] Skipping data.zip
If you want to download the file in another location, please specify a different path
[Already exists] Skipping enwiki-20230401.db
If you want to download the file in another location, please specify a different path
--2024-12-12 05:52:37--  https://raw.githubusercontent.com/shmsw25/FActScore/main/roberta_stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 870 [text/plain]
Saving to: ‘roberta_stopwords.txt.1’

roberta_stopwords.t 100%[===================>]     870  --.-KB/s    in 0s      

2024-12-12 05:52:38 (65.0 MB/s) - ‘roberta_stopwords.txt.1’ saved [870/870]

mv: cannot stat 'demos': No s

In [ ]:
fs2 = FactScorer(model_name="retrieval+ChatGPT",
                 data_dir="/factscore_data",
                 model_dir="/factscore_data",
                 cache_dir="/factscore_data",
                 openai_key="key.txt",
                 cost_estimate="consider_cache",
                 abstain_detection_type=None,
                 batch_size=256)

## Registering Knowledge Source

To do the factuality analysis on the respones of the CommunityLM responses, we first need to register them as a knowledge source. We register four knowledge sources based on:
1. Responses for Democratic politicians using the Democratic model
2. Responses for Republican politicians using the Democratic model
3. Responses for Democratic politicians using the Republican model
4. Responses for Republican politicians using the Republican model

The following code splits the responses of the Democratic model into responses for Democratic politicians and Republican politicians. Then it registers a knowledge source using the responses for Democratic politicians using the Democratic model. The same process can be applied for the other knowledge sources.



In [5]:
dem_data = []
rep_data = []

with open('democratic-final.csv', 'r') as file:     # Responses of the Democratic model for politicians of both parties
   reader = csv.reader(file)
   next(reader)  # Skip header
   for row in reader:
       party, politician, response = row
       entry = {
           "title": politician.strip('*'),
           "text": response
       }
       if 'Democrats' in party:
           dem_data.append(entry)
       elif 'Republicans' in party:
           rep_data.append(entry)

# Write Democrats JSONL
with open('democrats-democratic-final-model.jsonl', 'w') as f:    #jsonl for knowledge source of democratic politicians
   for entry in dem_data:
       f.write(json.dumps(entry) + '\n')

# Write Republicans JSONL
with open('republicans-democratic-final-model.jsonl', 'w') as f:  #jsonl for knowledge source of republican politicians
   for entry in rep_data:
       f.write(json.dumps(entry) + '\n')

In [ ]:
# Knowledge source using responses of Democratic model for Democratic politicians

fs2.register_knowledge_source("democrats-democratic-final-model",
                             data_path="democrats-democratic-final-model.jsonl",
                             db_path="democrats-democratic-final-model.db")

In [ ]:
# Knowledge source using responses of Democratic model for Democratic politicians

fs2.register_knowledge_source("republicans-democratic-final-model",
                             data_path="republicans-democratic-final-model.jsonl",
                             db_path="republicans-democratic-final-model.db")

### FactScore Calculations

To perform the factuality analysis, we calculate the FactScore of both models with respect to politicians of both parties. The code below is an example of how to calulate the FactScore of the Democratic model for Democratic and Republican politicians.

In [1]:
def get_model_data(model):
  if model == "democratic":
    df = pd.read_csv("democratic-factscore-calc.csv")  # Responses of the Democratic model for politicians of both parties
  else:
    df = pd.read_csv("republican-factscore-calc.csv")  # Responses of the Republican model for politicians of both parties
  return df

In [5]:
# choose the CommunityLM model for doing the analysis

model="democratic"  # or republican
df = get_model_data(model)
print(df.head())

       Party Politician                                           Response
0  Democrats  Joe Biden  Joe Biden is a lying , homophobic , misogynist...
1  Democrats  Joe Biden  Joe Biden is a terrible choice .\nIf trump is ...
2  Democrats  Joe Biden  Joe Biden is a liar , a con man , a traitor , ...
3  Democrats  Joe Biden  Joe Biden is a sexist :face_with_rolling_eyes:...
4  Democrats  Joe Biden  Joe Biden is a great President , and @USER is ...


In [4]:
democratic_topics = ["Joe Biden", "Bernie Sanders", "Elizabeth Warren", "Chuck Schumer", "Kamala Harris"]

republican_topics = ["Donald Trump", "Ron DeSantis", "John McCain", "Ted Cruz", "Nikki Haley"]

In [ ]:
democratic_responses = []

for topic in democratic_topics:
  responses = df.loc[df['Politician'].str.strip().str.lower() == topic.lower(), 'Response']
  combined = ' '.join(responses.head(20).dropna())
  democratic_responses.append(combined)

In [ ]:
republican_responses = []

for topic in republican_topics:
  responses = df.loc[df['Politician'].str.strip().str.lower() == topic.lower(), 'Response']
  combined = ' '.join(responses.head(20).dropna())
  republican_responses.append(combined)

In [ ]:
democratic_outputs = fs2.get_score(democratic_topics, democratic_responses, knowledge_source="democrats-democratic-final-model")
print (democratic_outputs["score"]) # FActScore
print (democratic_outputs["num_facts_per_response"]) # average number of atomic facts per response

In [ ]:
republican_outputs = fs2.get_score(republican_topics, republican_responses, knowledge_source="republicans-democratic-final-model")
print (republican_outputs["score"]) # FActScore
print (republican_outputs["num_facts_per_response"]) # average number of atomic facts per response